In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from tqdm.notebook import tqdm, trange

In [2]:
%load_ext autoreload
%autoreload 2

from hex_show import *
from hex_pathfinding import *
from hex_helpers import *
from hex_minmax import *
from hex_train_helpers import *

bsize=13
N=13

### Selfplay

In [3]:
from hex_transfer import *
from hex_net13x13 import *
net = Net13x13()

In [4]:
from hex_minmax import *
from hex_nnet import *
from hex_mcts import *

bsize = 13
N = 13


import pickle

boards = None
def selfplay_batched(ai, nstates=1000, frame=0, verbose=0):
    
    global boards
    bnum = ai.nparallel
    game_records = []
    
    ai.eta = 0.3

    # Iterator with tqdm??
    if verbose>=1:
        pbar = tqdm(total=nstates)
        
        
    def newboard():
        part = np.random.rand()
        while True:
            board = filledboard(bsize, part, frame=frame)
            if not winner(board):
                return board
            part *= .99 # avoid infinite loop

            
    if type(boards) is type(None):
        boards = [ newboard() for _ in range(bnum) ]
    turns = [ getturn(brd) for brd in boards ]
    records = [ [] for _ in range(bnum) ]
    
    
    #completedgames = 0
    #while completedgames < ngames:
    while len(game_records) < nstates:
        
        check_new_model(net, 'net_temp.dat')
        
        moves = ai.findmove(boards)
        for b in range(bnum):
            turn = turns[b]
            x, y = moves[b]
            records[b] += [(boards[b].copy(), (x, y), turn)] if turn > 0 else \
                          [(-boards[b].T.copy(), (y, x), turn)]
            boards[b][x, y] = turns[b]
            turns[b] *= -1
            
            won = winner(boards[b])
            if won:
                if verbose>=1:
                    pbar.update(len(records[b]))
                game_records += [ (b, m, t*won) for (b, m, t) in records[b] ] ####
                ##game_records += [ (b, m, rollout(b,n=10)) for (b, m, t) in records[b] ] ##
                #completedgames += 1
                records[b] = []
                boards[b] = newboard()
                turns[b] = getturn(boards[b])
        if np.random.rand() < 1/5:
            ai.clear()
    
    if verbose>=1:
        pbar.close()
        
    return game_records

In [ ]:
FRAME = 0

while True:
    newgames = selfplay_batched(BatchMCTS(nparallel=50, nsearches=100, net=net, params={'useQ': True, 'cuncert': 4}), nstates=100, frame=FRAME, verbose=1)
    save_selfplay(newgames)

Loaded new model parameters at time 02:01:57!

Saved 102 boards to temp/selfplay31.dat.
